##### Copyright 2019 The TensorFlow Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# pandas.DataFrame 로드

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/pandas_dataframe"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/pandas_dataframe.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

이 튜토리얼은 pandas 데이터 프레임을 `tf.data.Dataset`에 로드하는 방법의 예제를 제공합니다.

이 튜토리얼에서는 Cleveland Clinic Foundation for Heart Disease에서 제공하는 작은 [데이터세트를](https://archive.ics.uci.edu/ml/datasets/heart+Disease) 사용합니다. CSV에는 수백 개의 행이 있습니다. 각 행은 환자를 설명하고 각 열은 속성을 설명합니다. 이 정보를 사용하여 이 데이터세트에서 환자가 심장 질환이 있는지 여부를 예측하는 이진 분류 작업입니다.

## pandas를 사용하여 데이터 읽기

In [ ]:
import pandas as pd
import tensorflow as tf

심장 데이터세트가 포함된 csv 파일을 다운로드합니다.

In [ ]:
csv_file = tf.keras.utils.get_file('heart.csv', 'https://storage.googleapis.com/applied-dl/heart.csv')

pandas를 사용하여 csv 파일을 읽습니다.

In [ ]:
df = pd.read_csv(csv_file)

In [ ]:
df.head()

In [ ]:
df.dtypes

데이터 프레임의 `object` `thal` 열을 이산 숫자 값으로 변환합니다.

In [ ]:
df['thal'] = pd.Categorical(df['thal'])
df['thal'] = df.thal.cat.codes

In [ ]:
df.head()

## `tf.data.Dataset`를 사용하여 데이터 로드하기

`tf.data.Dataset.from_tensor_slices`를 사용하여 pandas 데이터 프레임에서 값을 읽습니다.

`tf.data.Dataset`를 사용할 때의 이점 중 하나는 간단하고 효율적인 데이터 파이프라인을 작성할 수 있다는 것입니다. 자세한 내용은 [데이터 로드 가이드](https://www.tensorflow.org/guide/data)를 참조하세요.

In [ ]:
target = df.pop('target')

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))

In [ ]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

`pd.Series`는 `__array__` 프로토콜을 구현하므로 `np.array` 또는 `tf.Tensor`를 사용하는 거의 모든 곳에서 투명하게 사용할 수 있습니다.

In [ ]:
tf.constant(df['thal'])

데이터세트를 섞고 배치 처리합니다.

In [ ]:
train_dataset = dataset.shuffle(len(df)).batch(1)

## 모델 생성 및 훈련

In [ ]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [ ]:
model = get_compiled_model()
model.fit(train_dataset, epochs=15)

## 특성 열의 대안

사전을 모델에 대한 입력으로 전달하는 것은 `tf.keras.layers.Input` 레이어의 일치하는 사전을 작성하는 것만큼 간편하며, [함수형 API](../../guide/keras/functional.ipynb)를 사용하여 필요한 사전 처리를 적용하고 레이어를 쌓습니다. [특성 열](../keras/feature_columns.ipynb)의 대안으로 사용할 수 있습니다.

In [ ]:
inputs = {key: tf.keras.layers.Input(shape=(), name=key) for key in df.keys()}
x = tf.stack(list(inputs.values()), axis=-1)

x = tf.keras.layers.Dense(10, activation='relu')(x)
output = tf.keras.layers.Dense(1)(x)

model_func = tf.keras.Model(inputs=inputs, outputs=output)

model_func.compile(optimizer='adam',
                   loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                   metrics=['accuracy'])

`tf.data`와 함께 사용했을 때 `pd.DataFrame`의 열 구조를 유지하는 가장 쉬운 방법은 `pd.DataFrame`을 `dict`로 변환하고 해당 사전을 조각화하는 것입니다.

In [ ]:
dict_slices = tf.data.Dataset.from_tensor_slices((df.to_dict('list'), target.values)).batch(16)

In [ ]:
for dict_slice in dict_slices.take(1):
  print (dict_slice)

In [ ]:
model_func.fit(dict_slices, epochs=15)